In [6]:
import numpy as np
import pandas as pd
from copy import deepcopy
from math import ceil, floor

pd.set_option("display.max_row", 0)
pd.set_option("display.max_column", 0)

### Dynamic Programming Solver

In [7]:
import copy

def knapsack_solver(weights:list, values:list, max_weight:int, top_n:int=1) -> list:
    if len(weights) == 0:
        return 0

    last_array = [-1 for _ in range(max_weight + 1)]
    last_path = [[] for _ in range(max_weight + 1)]

    for i in range(len(weights[0])):
        if weights[0][i] != max_weight:
            if last_array[weights[0][i]] < values[0][i]:
                last_array[weights[0][i]] = values[0][i]
                last_path[weights[0][i]] = [(0, i, weights[0][i])]

    for i in range(1, len(weights)):
        current_array = [-1 for _ in range(max_weight + 1)]
        current_path = [[] for _ in range(max_weight + 1)]

        for j in range(len(weights[i])):
            for k in range(weights[i][j], max_weight + 1):
                if last_array[k - weights[i][j]] > 0:
                    if current_array[k] < last_array[k - weights[i][j]] + values[i][j]:
                        current_array[k] = last_array[k - weights[i][j]] + values[i][j]
                        current_path[k] = deepcopy(last_path[k - weights[i][j]])
                        current_path[k].append((i, j, weights[i][j]))

        last_array = current_array
        last_path = current_path

    solution = [elem for elem in zip(last_array, last_path) if elem[0] != -1]
    solution = sorted(solution, key=lambda tup: tup[0], reverse=True)

    return solution[:top_n]

### Answer Formatter

In [8]:
def solve(surveys:list, max_extension:int, max_manpower:int, top_n:int=1) -> dict:
    daily_agents = lambda c, r, d, x: ceil(c / (r * (d + x)))

    problem_dict = [dict.fromkeys([]) for _ in surveys]
    for idx, (cr, rate, day) in enumerate(surveys):
        for extension in range(max_extension):
            agents = daily_agents(cr, rate, day, extension+1)
            if agents not in problem_dict[idx].values():
                problem_dict[idx][max_extension-extension] = agents

    values = []
    weights = []
    for survey in problem_dict:
        values.append(tuple(survey.keys()))
        weights.append(tuple(survey.values()))

    solution = knapsack_solver(weights, values, max_manpower, top_n)
    viable_days = [list(d.keys()) for d in problem_dict]
    schedule = []

    for idx, (score, path) in enumerate(solution):
        agent = [tup[2] for tup in path]
        extend = [max_extension-viable_days[idx][elem] for idx, elem in enumerate([tup[1] for tup in path])]
        schedule.append((agent, extend))

    if schedule:
        for i in range(top_n):
            print(
                [
                    agent*rate
                    for agent, (_, rate, _)
                    in zip(schedule[i][0], surveys)
                ]
            )
        return schedule
    else:
        print(solution)
        print("Solution not found. Either extend the maximum extension or increase manpower.")
        return

### Example

In [10]:
surveys = [(300, 15, 0), (1100, 12, 0), (700, 14, 0), (900, 9, 0), (400, 20, 0), (600, 18, 0), (700, 17, 0)]
max_extension = 14
max_manpower = 40
solve(surveys, max_extension, max_manpower, 10)

[60, 96, 70, 72, 80, 90, 102]
[45, 96, 70, 72, 80, 90, 102]
[45, 84, 70, 72, 80, 90, 102]
[45, 84, 70, 72, 60, 90, 102]
[45, 84, 70, 72, 60, 72, 102]
[45, 84, 70, 72, 60, 72, 85]
[45, 84, 70, 72, 60, 72, 68]
[30, 84, 70, 72, 60, 72, 68]
[30, 84, 56, 72, 60, 72, 68]
[30, 84, 56, 72, 40, 72, 68]


[([4, 8, 5, 8, 4, 5, 6], [4, 11, 9, 12, 4, 6, 6]),
 ([3, 8, 5, 8, 4, 5, 6], [6, 11, 9, 12, 4, 6, 6]),
 ([3, 7, 5, 8, 4, 5, 6], [6, 13, 9, 12, 4, 6, 6]),
 ([3, 7, 5, 8, 3, 5, 6], [6, 13, 9, 12, 6, 6, 6]),
 ([3, 7, 5, 8, 3, 4, 6], [6, 13, 9, 12, 6, 8, 6]),
 ([3, 7, 5, 8, 3, 4, 5], [6, 13, 9, 12, 6, 8, 8]),
 ([3, 7, 5, 8, 3, 4, 4], [6, 13, 9, 12, 6, 8, 10]),
 ([2, 7, 5, 8, 3, 4, 4], [9, 13, 9, 12, 6, 8, 10]),
 ([2, 7, 4, 8, 3, 4, 4], [9, 13, 12, 12, 6, 8, 10]),
 ([2, 7, 4, 8, 2, 4, 4], [9, 13, 12, 12, 9, 8, 10])]